## SAY SOMETHING HERE

A company-year long format xt panel data table, 2005-2016 
 n = 287,829 observations (46,412 firms)
	(third as many companies in 2016 as in earlier years)
 - ID variables	comp_id	(numerical) company identifier
- year	(calendar) year

 other important variables
 
- exit_year	year of exit (missing if still in business on 2016.12.31) 
- sales		sales in year (EUR)
- curr_assets	current assets at end of year (EUR)
- ind2		2-digit NACE industry code

DEFAULT =  Existed in 2014 (sales > 0), but did not exists in 2015 (sales is 0 or missing)

In [230]:
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn.metrics import brier_score_loss, roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [306]:
data = pd.read_csv('cs_bisnode_panel.zip')
data

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
0,1.001034e+06,2005-01-01,2005-12-31,NaN,692.592590,7266.666504,7574.074219,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
1,1.001034e+06,2006-01-01,2006-12-31,NaN,603.703674,13122.222656,12211.111328,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
2,1.001034e+06,2007-01-01,2007-12-31,NaN,425.925934,8196.295898,7800.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
3,1.001034e+06,2008-01-01,2008-12-31,NaN,300.000000,8485.185547,7781.481445,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,NaN
4,1.001034e+06,2009-01-01,2009-12-31,NaN,207.407410,5137.037109,15300.000000,0.0,0.0,0.0,...,mix,Domestic,5630.0,56.0,3.0,1,Central,1990-11-19,NaN,0.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287824,4.641209e+11,2011-01-01,2011-12-31,NaN,0.000000,1807.407349,3.703704,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287825,4.641209e+11,2012-01-01,2012-12-31,NaN,0.000000,1518.518555,96.296295,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287826,4.641209e+11,2013-01-01,2013-12-31,NaN,0.000000,988.888916,77.777779,0.0,0.0,0.0,...,NaN,NaN,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN
287827,4.641209e+11,2014-01-01,2014-12-31,NaN,0.000000,644.444458,233.333328,0.0,0.0,0.0,...,female,Domestic,5510.0,55.0,3.0,2,East,1990-08-22,NaN,NaN


In [307]:
data = data.loc[data['ind2'] == 26]
data
#data for specific industry

,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,gender,origin,nace_main,ind2,ind,urban_m,region_m,founded_date,exit_date,labor_avg
960,6.538183e+06,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
961,6.538183e+06,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
962,6.538183e+06,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
963,6.538183e+06,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
964,6.538183e+06,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,male,Foreign,2630.0,26.0,2.0,2,East,1992-08-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287439,4.628231e+11,2012-01-01,2012-12-31,NaN,359.259247,21662.962891,19481.482422,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.333333
287440,4.628231e+11,2013-01-01,2013-12-31,NaN,3159.259277,15488.888672,21440.740234,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.298611
287441,4.628231e+11,2014-01-01,2014-12-31,NaN,2581.481445,70059.257812,74911.109375,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.486111
287442,4.628231e+11,2015-01-01,2015-12-31,NaN,2177.777832,46062.964844,49844.445312,0.000000,0.0,0.000000,...,male,Domestic,2630.0,26.0,2.0,1,Central,2009-12-06,NaN,0.562500


In [319]:
#creating y variable ('default')
# generate status_alive; if sales larger than zero and not-NA, then firm is alive
data["status_alive"] = (data["sales"] > 0 & (False == data["sales"].isna())).astype(int)

# defaults in 1 year if there are sales in this year but no sales one year later
# Status_in_one_year: data.groupby('comp_id')['status_alive'].shift(-1)
data["default"] = (
    (data["status_alive"] == 1)
    & (data.groupby("comp_id")["status_alive"].shift(-1) == 0)
).astype(int)

In [311]:
#cleaning dataset:

#x = amort, curr_assets, curr_liab, extra_exp, extra_inc, fixed_assets, inc_bef_tax, intang_assets, inventories
#liq_assets, material_exp, personnel_exp, profit_loss_year, share_eq, subscribed_cap, tang_assets
#ceo_count, foreign (%), female(%), inoffice_days, *gender(female_only, male_only, mixed_gender),
#*origin(domestic, foreign, mix), urban_m (capital city, big_city, other_hq_city), *region_m (Central, East, West)


#imputing missing amort values with median (less affected by outliers)
#creating flag_variable for missing values and imputed values
data.amort.isnull().sum()
data['amort_missing_flag'] = data.amort.isnull().astype(int)
data['amort_imputed_flag'] = data.amort.isnull().astype(int)
data.fillna({'amort':data['amort'].median()}, inplace = True)

#imputing missing curr_assets values with median
#creating flag_variable for missing values and imputed values
data.curr_assets.isnull().sum()
data['curr_assets_missing_flag'] = data.curr_assets.isnull().astype(int)
data['curr_assets_imputed_flag'] = data.curr_assets.isnull().astype(int)
data.fillna({'curr_assets':data['curr_assets'].median()}, inplace = True)

#imputing missing curr_liab values with median
#creating flag_variable for missing values and imputed values
data.curr_liab.isnull().sum()
data['curr_liab_missing_flag'] = data.curr_liab.isnull().astype(int)
data['curr_liab_imputed_flag'] = data.curr_liab.isnull().astype(int)
data.fillna({'curr_liab':data['curr_liab'].median()}, inplace = True)

#imputing missing extra_exp values with median
#creating flag_variable for missing values and imputed values
data.extra_exp.isnull().sum()
data['extra_exp_missing_flag'] = data.extra_exp.isnull().astype(int)
data['extra_exp_imputed_flag'] = data.extra_exp.isnull().astype(int)
data.fillna({'extra_exp':data['extra_exp'].median()}, inplace = True)

#imputing missing extra_inc values with median
#creating flag_variable for missing values and imputed values
data.extra_inc.isnull().sum()
data['extra_inc_missing_flag'] = data.extra_inc.isnull().astype(int)
data['extra_inc_imputed_flag'] = data.extra_inc.isnull().astype(int)
data.fillna({'extra_inc':data['extra_inc'].median()}, inplace = True)

#delete finished_prof column (too many null values): 13054/14877
data = data.drop('finished_prod', axis=1)

#imputing missing fixed_assets values with median
#creating flag_variable for missing values and imputed values
data.fixed_assets.isnull().sum()
data['fixed_assets_missing_flag'] = data.fixed_assets.isnull().astype(int)
data['fixed_assets_imputed_flag'] = data.fixed_assets.isnull().astype(int)
data.fillna({'fixed_assets':data['fixed_assets'].median()}, inplace = True)

#imputing missing inc_bef_tax values with median
#creating flag_variable for missing values and imputed values
data.inc_bef_tax.isnull().sum()
data['inc_bef_tax_missing_flag'] = data.inc_bef_tax.isnull().astype(int)
data['inc_bef_tax_imputed_flag'] = data.inc_bef_tax.isnull().astype(int)
data.fillna({'inc_bef_tax':data['inc_bef_tax'].median()}, inplace = True)

#imputing missing intang_assets values with median
#creating flag_variable for missing values and imputed values
data.intang_assets.isnull().sum()
data['intang_assets_missing_flag'] = data.intang_assets.isnull().astype(int)
data['intang_assets_imputed_flag'] = data.intang_assets.isnull().astype(int)
data.fillna({'intang_assets':data['intang_assets'].median()}, inplace = True)

#imputing missing inventories values with median
#creating flag_variable for missing values and imputed values
data.inventories.isnull().sum()
data['inventories_missing_flag'] = data.inventories.isnull().astype(int)
data['inventories_imputed_flag'] = data.inventories.isnull().astype(int)
data.fillna({'inventories':data['inventories'].median()}, inplace = True)

#imputing missing liq_assets values with median
#creating flag_variable for missing values and imputed values
data.liq_assets.isnull().sum()
data['liq_assets_missing_flag'] = data.liq_assets.isnull().astype(int)
data['liq_assets_imputed_flag'] = data.liq_assets.isnull().astype(int)
data.fillna({'liq_assets':data['liq_assets'].median()}, inplace = True)

#imputing missing material_exp values with median
#creating flag_variable for missing values and imputed values
data.material_exp.isnull().sum()
data['material_exp_missing_flag'] = data.material_exp.isnull().astype(int)
data['material_exp_imputed_flag'] = data.material_exp.isnull().astype(int)
data.fillna({'material_exp':data['material_exp'].median()}, inplace = True)

#delete net_dom_sales column (too many null values): 13022/14877
data = data.drop('net_dom_sales', axis=1)

#delete net_exp_sales column (too many null values): 13022/14877
data = data.drop('net_exp_sales', axis=1)

#imputing missing personnel_exp values with median
#creating flag_variable for missing values and imputed values
data.personnel_exp.isnull().sum()
data['personnel_exp_missing_flag'] = data.personnel_exp.isnull().astype(int)
data['personnel_exp_imputed_flag'] = data.personnel_exp.isnull().astype(int)
data.fillna({'personnel_exp':data['personnel_exp'].median()}, inplace = True)

#imputing missing profit_loss_year values with median
#creating flag_variable for missing values and imputed values
data.profit_loss_year.isnull().sum()
data['profit_loss_year_missing_flag'] = data.profit_loss_year.isnull().astype(int)
data['profit_loss_year_imputed_flag'] = data.profit_loss_year.isnull().astype(int)
data.fillna({'profit_loss_year':data['profit_loss_year'].median()}, inplace = True)

#imputing missing share_eq values with median
#creating flag_variable for missing values and imputed values
data.share_eq.isnull().sum()
data['share_eq_year_missing_flag'] = data.share_eq.isnull().astype(int)
data['share_eq_year_imputed_flag'] = data.share_eq.isnull().astype(int)
data.fillna({'share_eq':data['share_eq'].median()}, inplace = True)

#imputing missing subscribed_cap values with median
#creating flag_variable for missing values and imputed values
data.subscribed_cap.isnull().sum()
data['subscribed_cap_missing_flag'] = data.subscribed_cap.isnull().astype(int)
data['subscribed_cap_imputed_flag'] = data.subscribed_cap.isnull().astype(int)
data.fillna({'subscribed_cap':data['subscribed_cap'].median()}, inplace = True)

#imputing missing tang_assets values with median
#creating flag_variable for missing values and imputed values
data.tang_assets.isnull().sum()
data['tang_assets_missing_flag'] = data.tang_assets.isnull().astype(int)
data['tang_assets_imputed_flag'] = data.tang_assets.isnull().astype(int)
data.fillna({'tang_assets':data['tang_assets'].median()}, inplace = True)

#delete wages column (too many null values): 13060/14877
data = data.drop('wages', axis=1)

#imputing missing ceo_count values with median
#creating flag_variable for missing values and imputed values
data.ceo_count.isnull().sum()
data['ceo_count_missing_flag'] = data.ceo_count.isnull().astype(int)
data['ceo_count_imputed_flag'] = data.ceo_count.isnull().astype(int)
data.fillna({'ceo_count':data['ceo_count'].median()}, inplace = True)

#imputing missing foreign values with median (% of foreign CEOs)
#creating flag_variable for missing values and imputed values
data.rename(columns= {'foreign':'foreign_ceo'}, inplace = True)
data.foreign_ceo.isnull().sum()
data['foreign_ceo_missing_flag'] = data.foreign_ceo.isnull().astype(int)
data['foreign_ceo_imputed_flag'] = data.foreign_ceo.isnull().astype(int)
data.fillna({'foreign_ceo':data['foreign_ceo'].median()}, inplace = True)

#imputing missing female values with median (% of female CEOs)
#creating flag_variable for missing values and imputed values
data.female.isnull().sum()
data['female_missing_flag'] = data.female.isnull().astype(int)
data['female_imputed_flag'] = data.female.isnull().astype(int)
data.fillna({'female':data['female'].median()}, inplace = True)

#imputing missing inoffice_days values with median (could use median or mean in this case as distribution is not skewed- using median for consistency)
#creating flag_variable for missing values and imputed values
data.inoffice_days.isnull().sum()
data['inoffice_days_missing_flag'] = data.inoffice_days.isnull().astype(int)
data['inoffice_days_imputed_flag'] = data.inoffice_days.isnull().astype(int)
data.fillna({'inoffice_days':data['inoffice_days'].median()}, inplace = True)

#impute the nan gender values with mode (categorical variable)
data['gender'] = data['gender'].fillna(data['gender'].mode()[0])
#categorical variable gender (female only, male only, or mixed)
data['female_only'] = (data['gender'] == 'female').astype(int)
data['male_only'] = (data['gender'] == 'male').astype(int)
data['mixed_gender'] = (data['gender'] == 'mix').astype(int)

#impute the nan origin values with mode (categorical variable)
data['origin'] = data['origin'].fillna(data['origin'].mode()[0])
#categorical variable origin (domestic, foreign, mix)
data['domestic'] = (data['origin'] == 'Domestic').astype(int)
data['foreign'] = (data['origin'] == 'foreign').astype(int)
data['mix'] = (data['origin'] == 'mix').astype(int)

#impute the nan urban_m values with mode (categorical variable)
data['urban_m'] = data['urban_m'].fillna(data['urban_m'].mode()[0])
#categorical variable urban_m (1- capital city, 2-other big city, 3-other)
data['capital_city'] = (data['urban_m'] == 1).astype(int)
data['big_city'] = (data['urban_m'] == 2).astype(int)
data['other_hq_city'] = (data['urban_m'] == 3).astype(int)

#impute the nan region_m values with mode (categorical variable)
data['region_m'] = data['region_m'].fillna(data['region_m'].mode()[0])
#categorical variable region_m (Central, East, West)
data['central'] = (data['region_m'] == 'Central').astype(int)
data['east'] = (data['region_m'] == 'East').astype(int)
data['west'] = (data['region_m'] == 'West').astype(int)

In [312]:
variables = ['amort', 'curr_assets', 'curr_liab', 'extra_exp', 'extra_inc', 'fixed_assets', 'inc_bef_tax', 
             'intang_assets', 'inventories','liq_assets', 'material_exp', 'personnel_exp', 'profit_loss_year',
             'share_eq', 'subscribed_cap', 'tang_assets', 'ceo_count', 'foreign_ceo', 'female', 
             'inoffice_days', 'female_only', 'male_only', 'domestic', 'foreign', 'capital_city','other_hq_city',
             'east', 'west']
#reference group for gender is mixed_gender firms
#refrence group for origin is mix CEO origin
#reference group for urban_m is big_city HQ location
#reference group for region_m is Central HQ region

### Creating holdout and training samples

In [313]:
#criteria for the holdout sample so I can mask the training sample with !holdout_criteria
holdout_criteria = (data['year'] == 2014) & ((data['sales'] >=1000) & (data['sales']<=10000000))

In [314]:
#holdout sample creation
holdout = data[holdout_criteria]
print(f'Holdout shape: {holdout.shape}')
holdout.head()

Holdout shape: (1037, 98)


,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,mixed_gender,domestic,foreign,mix,capital_city,big_city,other_hq_city,central,east,west
969,6538183.0,2014-01-01,2014-12-31,NaN,1018.518494,17022.222656,3040.740723,0.0,0.0,0.0,...,0,1,0,0,0,1,0,0,1,0
1128,8416055.0,2014-01-01,2014-12-31,NaN,425.925934,105740.742188,6918.518555,0.0,0.0,0.0,...,0,1,0,0,1,0,0,1,0,0
1467,12428378.0,2014-01-01,2014-12-31,NaN,0.000000,0.000000,9277.777344,0.0,0.0,0.0,...,0,1,0,0,0,1,0,1,0,0
1706,17776540.0,2014-01-01,2014-12-31,NaN,48.148148,142296.296875,164237.031250,0.0,0.0,0.0,...,1,1,0,0,1,0,0,1,0,0
1735,18626760.0,2014-01-01,2014-12-31,NaN,5755.555664,95185.187500,45766.667969,0.0,0.0,0.0,...,0,1,0,0,0,1,0,0,0,1


In [318]:
holdout.default.value_counts()
#no matter what I do I get 26 companies defaulted...so, sticking with 26 firms defaulted for now

default
0    1011
1      26
Name: count, dtype: int64

In [316]:
training = data[~holdout_criteria]
training = training.loc[training['year'] != 2016]
print(f'Training shape: {training.shape}')
training.head()

Training shape: (13252, 98)


,comp_id,begin,end,COGS,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,mixed_gender,domestic,foreign,mix,capital_city,big_city,other_hq_city,central,east,west
960,6538183.0,2005-01-01,2005-12-31,NaN,792.592590,6237.037109,348.148163,0.000000,0.0,0.000000,...,0,0,0,0,0,1,0,0,1,0
961,6538183.0,2006-01-01,2006-12-31,NaN,196.296295,6007.407227,451.851837,0.000000,0.0,0.000000,...,0,0,0,0,0,1,0,0,1,0
962,6538183.0,2007-01-01,2007-12-31,NaN,233.333328,9007.407227,2637.037109,0.000000,0.0,0.000000,...,0,0,0,0,0,1,0,0,1,0
963,6538183.0,2008-01-01,2008-12-31,NaN,422.222229,7707.407227,1651.851807,0.000000,0.0,0.000000,...,0,0,0,0,0,1,0,0,1,0
964,6538183.0,2009-01-01,2009-12-31,NaN,1525.925903,18196.296875,8714.814453,18.518518,0.0,-18.518518,...,0,0,0,0,0,1,0,0,1,0


In [317]:
training.default.value_counts()

default
0    12898
1      354
Name: count, dtype: int64

## Model 1: Simple Linear Logistic Regression (Logit)

In [225]:
#created some interaction terms by using ggplot to see possible interactions:
data['interaction_curr_assets_vs_curr_liab'] = data['curr_assets'] * data['curr_liab']
data['interaction_inoffice_days_vs_foreign_ceo'] = data['inoffice_days'] * data['foreign_ceo']

interaction_terms = ['interaction_curr_assets_vs_curr_liab', 'interaction_inoffice_days_vs_foreign_ceo']

In [226]:
basevars = data[variables]
base_interactions = data[interaction_terms]
X1 = pd.concat([basevars, base_interactions], axis=1)

In [227]:
ols_modelx1 = LinearRegression().fit(X1, y)

In [231]:
#function from ch17 Jp Nb to get regression results
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [232]:
regression_results(y, ols_modelx1.predict(X1))

explained_variance:  0.0077
r2:  0.0077
MAE:  0.082
MSE:  0.0405
RMSE:  0.2014


In [267]:
#leaves it 1-D
ols_modelx1.coef_.flatten().shape

#function to create a matrix showing coefficients for each variable
def create_coef_matrix(X, model):
    coef_matrix = pd.concat(
        [pd.DataFrame(X.columns),pd.DataFrame(model.coef_.flatten())], axis = 1
    )
    coef_matrix.columns = ['variable', 'coefficient']
    coef_matrix.iloc[-1] = ['Intercept', model.intercept_.flatten()[0]]
    return coef_matrix

In [268]:
#using linear regression to make the Logit regression
glm_modelx1 = LogisticRegression(
    solver="newton-cg",max_iter=1000, penalty="none", random_state = 20240223).fit(X1, y)
regression_results(y, glm_modelx1.predict(X1))
#RMSE is 0.2135 me

explained_variance:  -0.0789
r2:  -0.1153
MAE:  0.0456
MSE:  0.0456
RMSE:  0.2135


In [265]:
create_coef_matrix(X1, glm_modelx1)

,variable,coefficient
0,amort,-2.989335e-22
1,curr_assets,-2.989397e-21
2,curr_liab,-1.740425e-21
3,extra_exp,-1.284643e-23
4,extra_inc,-4.110820e-23
5,fixed_assets,-3.365272e-21
6,inc_bef_tax,-3.236584e-22
7,intang_assets,-3.420265e-22
8,inventories,-9.054427e-22
9,liq_assets,-6.418255e-22
